<a href="https://colab.research.google.com/github/AI-Sunmoon2021/model_vgg16/blob/main/vgg16_modify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, Input
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from tensorflow import keras
from tensorflow.keras import optimizers
from os import listdir

In [ ]:
#이미지 사이즈의 정의
image_size = 50

#listdir('./xx/')파일 안에 있는 이미지를 넣는다.
path_bh = [filename for filename in listdir('/content/drive/MyDrive/train/rare/') if not filename.startswith('.')]
path_bu = [filename for filename in listdir('/content/drive/MyDrive/train/welldone/') if not filename.startswith('.')]


#각 파일을 순서대로 호출한다. (파일에 있는 개수만큼 반복한다.)
# OpenCV로 화상 사이즈를 리사이즈 하여 통일한다.
#img_에 리사이즈 한 것을 더한다.

img_bh = []
img_bu = []

for i in range(len(path_bh)):
   img = cv2.imread('/content/drive/MyDrive/train/rare/'+ path_bh[i])
   img = cv2.resize(img,(image_size,image_size))
   img_bh.append(img)

for i in range(len(path_bu)):
   img = cv2.imread('/content/drive/MyDrive/train/welldone/'+ path_bu[i])
   img = cv2.resize(img,(image_size,image_size))
   img_bu.append(img)

X = np.array(img_bh + img_bu) #행렬을 잇다.
y = np.array([0]*len(img_bh) + [1]*len(img_bu)) #０、１、２하고 종류별로 번호를 매긴다.
rand_index = np.random.permutation(np.arange(len(X))) #np.arange(x)의 배열을 랜덤으로 정렬하여 rand_index 에 넣는다.
X = X[rand_index] #X를 rand_index 순으로 정렬한다
y = y[rand_index] #y를 rand_index순으로 정렬한다


#데이터를 분할한다.
# X_train = X[:int(len(X)*0.8)].astype(np.float32) # 트레이닝 데이터 80% 미만
# y_train = y[:int(len(y)*0.8)].astype(np.float32)
X_train = X[:int(len(X)*0.8)]## 트레이닝 데이터 80% 미만
y_train = y[:int(len(y)*0.8)]
X_test = X[int(len(X)*0.8):] #트레이닝 데이터 이외
y_test = y[int(len(y)*0.8):]

#각 속성을 보여준다.
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

y_train = to_categorical(y_train) #One-Hot벡터를 생성한다.
y_test = to_categorical(y_test) #One-Hot벡터를 생성한다.

(1318, 50, 50, 3)
(1318,)
(330, 50, 50, 3)
(330,)


In [ ]:
# 전이학습(VGG16모델)을 활용한다.ImageNet 로 사전 학습한 중량감을 읽어 들인다.
input_tensor = Input(shape=(image_size, image_size, 3)) 
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# 특징 추출 부분 이후에 새롭게 다른 층을 추가하기 위해, 미리 VGG와는 다른 모델(여기서는 top_model)을 정의한다.
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:])) #VGG를 결합한다.

top_model.add(Dense(256, activation="sigmoid"))#1번째 전체결합층의 출력유닛수는 256, 활성화함수는 shigmoid.
top_model.add(Dropout(0.5))                     #과학습을 막는다.
top_model.add(Dense(64, activation='sigmoid'))
top_model.add(Dropout(0.5))
top_model.add(Dense(32, activation='sigmoid'))
top_model.add(Dropout(0.5))
top_model.add(Dense(2, activation='softmax')) #소프트맥스 함수를 이용해 3차원으로 출력한다.Dense를 3로 하면 3차원이 된다.

In [13]:
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

for layer in model.layers[:15]:
   layer.trainable = False
  
model.compile(loss='categorical_crossentropy', #다클래스 분류의 손실 함수를 정의한다.
             optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), #최적화 함수를 정의한다.
             metrics=['accuracy'])#평가함수를 정의한다.

#X_train, y_train 을 이용해 학습을 실시한다.테스트 데이터는 X_test, y_test.
# history = model.fit(X_train, y_train, batch_size=32, epochs=75, verbose=1, validation_data=(X_test, y_test))
history = model.fit(X_train, y_train, batch_size=15, epochs=25, verbose=1, validation_data=(X_test, y_test))
# 정밀도 평가한다.
score = model.evaluate(X_test, y_test, batch_size=15, verbose=0)
print('validation loss:{0[0]}\nvalidation accuracy:{0[1]}'.format(score))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/25
88/88 [==============================] - 54s 603ms/step - loss: 0.6963 - accuracy: 0.5797 - val_loss: 0.5977 - val_accuracy: 0.6333
Epoch 2/25
88/88 [==============================] - 53s 602ms/step - loss: 0.6884 - accuracy: 0.5766 - val_loss: 0.5903 - val_accuracy: 0.6818
Epoch 3/25
88/88 [==============================] - 53s 602ms/step - loss: 0.6894 - accuracy: 0.5797 - val_loss: 0.5843 - val_accuracy: 0.7364
Epoch 4/25
88/88 [==============================] - 53s 602ms/step - loss: 0.6800 - accuracy: 0.5895 - val_loss: 0.5762 - val_accuracy: 0.7758
Epoch 5/25
88/88 [==============================] - 53s 603ms/step - loss: 0.6851 - accuracy: 0.5850 - val_loss: 0.5681 - val_accuracy: 0.8061
Epoch 6/25
88/88 [==============================] - 53s 602ms/step - loss: 0.6444 - accuracy: 0.6237 - val_loss: 0.5607 - val_accuracy: 0.8455
Epoch 7/25
88/88 [==============================] - 53s 603ms/step - loss: 0.6434 - accuracy: 0.6305 - val_loss: 0.5525 - val_accuracy: 0.8455

In [14]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test) 

print(classification_report(y_test.argmax(axis=1),y_pred.argmax(axis=1), target_names = ['rare', 'welldone']))

              precision    recall  f1-score   support

        rare       0.96      0.99      0.97       203
    welldone       0.98      0.93      0.95       127

    accuracy                           0.96       330
   macro avg       0.97      0.96      0.96       330
weighted avg       0.96      0.96      0.96       330

